In [1]:
import os
import sys

import numpy as np
import pandas as pd
import seaborn as sns

sys.path.append('/Users/fgu/dev/projects/entropy')
import entropy.helpers.aws as aws
import entropy.data.cleaners as cl
import entropy.data.helpers as dh

sns.set_style('whitegrid')
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 120)
pd.set_option('max_colwidth', None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

Notebook purpose

- Understand nature of duplicate transactions and explore solutions

### Summary

1. `['user_id', 'date', 'amount', 'account_id', 'desc']` are identical -> drop in main analysis.

2. `['user_id', 'date', 'amount', 'account_id']` are identical and `desc` is similar -> drop from main analysis. *

3. `['user_id', 'date', 'amount', 'account_id']` are identical and `desc` not similar -> keep in main analysis.

4. `['user_id', 'date', 'amount']`, `desc` may or may not differ, but `account_id` differs. This is relevant if there are (many) duplicated accounts, in which case a different account number is no guarantee for a different account. -> ignore for now, ask MDB to get list of duplicated accounts.


\* in this category we include cases where desc of one txn is `<mdbremoved>` while other isn't, so even though descriptions are not similar in this case, we assume that redaction masks available description.

In [4]:
df = pd.read_parquet('~/tmp/entropy_X77.parquet')

Helper functions

In [31]:
def distr(x):
    pcts = [.01, .05, .1, .25, .50, .75, .90, .95, .99]
    return x.describe(percentiles=pcts).round(2)

def dup_txn_sample(df, col_subset, n=100, seed=2312):
    """Draws sample of size n of duplicate txns as defined by col_subset."""
    dups = df[df.duplicated(subset=col_subset, keep=False)].copy()
    dups['group'] = dups.groupby(col_subset).ngroup()
    unique_groups = np.unique(dups.group)
    rng = np.random.default_rng(seed=seed)
    sample = rng.choice(unique_groups, size=n)
    return dups[dups.group.isin(sample)]

## Case studies

Below three case studies of duplicates

In [ ]:
dh.user_date_data(df, 35177, '1 Jan 2020')

In [ ]:
dh.user_date_data(df, 362977, '1 Jan 2020')

In [ ]:
dh.user_date_data(df, 467877, '1 Jan 2020')

## Type 1 duplicates

### Definition
- `['user_id', 'date', 'amount', 'account_id', 'desc']` are identical.
 
- This includes transactions where desc for both is `<mdbremoved>`, where we have to make a call whether or not to assume they are the same.

- Reasons for false positives (FP): user makes two identical transactions on the same day (or on subsequent days for txns that appear with a delay). Plausible cases are coffee and betting shop txns.

In [41]:
dup_subset = ['user_id', 'date', 'amount', 'account_id', 'desc']
dup_var = 'dup1'

df[dup_var] = df.duplicated(subset=dup_subset)

### Prevalence and value

How prevalent are duplicates?

In [8]:
n_df = len(df)
n_dups = len(df[df[dup_var]])
n_users_dups = df[df[dup_var]].user_id.nunique()
n_users_df = df.user_id.nunique()
txt = 'About {:.1%} of transactions across {:.0%} of users are potential dups.'
print(txt.format(n_dups / n_df, n_users_dups / n_users_df))

About 1.7% of transactions across 97% of users are potential dups.


Gross value of duplicated txns

In [12]:
gross_value = df[df[dup_var]].set_index('user_id').amount.abs().groupby('user_id').sum()
distr(gross_value)

count       415.00
mean       4459.53
std       14957.93
min           1.00
1%            4.54
5%           20.46
10%          61.97
25%         237.31
50%         830.10
75%        2647.96
90%        8980.22
95%       16434.12
99%       59013.14
max      183754.34
Name: amount, dtype: float64

Most frequent txns description

In [13]:
df[df[dup_var]].desc.value_counts(dropna=False)[:10]

<mdbremoved>                       1962
<mdbremoved>                        516
<mdbremoved> ft                     359
b365 moto                           263
paypal payment                      195
tfl travel charge tfl.gov.uk/cp     167
www.skybet.com cd 9317              165
<mdbremoved> so                     157
betfair.-purchase                   146
<mdbremoved> - s/o                  143
Name: desc, dtype: int64

Most frequent auto tag

In [14]:
df[df[dup_var]].tag_auto.value_counts(dropna=False)[:10]

NaN                         6326
transfers                   3038
gambling                    2273
enjoyment                   1617
public transport            1132
lunch or snacks             1019
bank charges                 862
entertainment, tv, media     556
cash                         520
dining or going out          507
Name: tag_auto, dtype: int64

Proportion of txns per auto tag that are duplicated

In [66]:
txns_per_tag_overall = df.tag_auto.value_counts(dropna=False)
txns_per_tag_duplicated = df[df[dup_var]].tag_auto.value_counts(dropna=False) 
p_dup_per_tag = (txns_per_tag_duplicated / txns_per_tag_overall)
p_dup_per_tag.sort_values(ascending=False)[:40]

repayments                       0.303867
legal                            0.175299
saving (general)                 0.125000
vet                              0.104603
investment - other               0.102564
gambling                         0.094977
media bundle                     0.088889
bank charges                     0.078101
social club                      0.076923
isa                              0.066845
vehicle                          0.066667
school fees                      0.066667
store card repayment             0.057554
payday loan funds                0.057143
cash                             0.051044
investments or shares            0.050505
supermarket                      0.050198
vehicle hire                     0.046154
cycling                          0.041162
bills                            0.040550
child - everyday or childcare    0.038734
hobbies or activities            0.037280
sports equipment                 0.036217
interest charges                 0

### Inspect dups

In [40]:
dup_txn_sample(df, dup_subset, n=2, seed=None)

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,savings,dup1,group
739537,494439523,2019-01-02,434077,-270.0,<mdbremoved> via online - pymt - dpc,NaN,None,None,False,e9 6,2018-04-06,NaN,1967.0,2018-04-06,928688,2020-03-11 09:41:00,natwest bank,current,2019-01-11,2019-02-04,False,7038.149902,NaN,NaN,NaN,NaN,u,201901,6662.920410,15507.500000,False,False,10141
739538,494439524,2019-01-02,434077,-270.0,<mdbremoved> via online - pymt - dpc,NaN,None,None,False,e9 6,2018-04-06,NaN,1967.0,2018-04-06,928688,2020-03-11 09:41:00,natwest bank,current,2019-01-11,2019-02-04,False,7038.149902,NaN,NaN,NaN,NaN,u,201901,6662.920410,15507.500000,False,True,10141
774834,510986520,2019-02-09,440877,2.3,"www.justpark.com, internet",NaN,spend,motor,False,w3 9,2018-05-23,70k to 80k,1965.0,2018-05-23,959817,2020-08-16 00:56:00,barclaycard,credit card,2019-02-13,1900-01-01,True,-28.580000,NaN,parking,NaN,parking,c,201902,575.950134,78606.601562,False,False,10719
774835,510986521,2019-02-09,440877,2.3,"www.justpark.com, internet",NaN,spend,motor,False,w3 9,2018-05-23,70k to 80k,1965.0,2018-05-23,959817,2020-08-16 00:56:00,barclaycard,credit card,2019-02-13,1900-01-01,True,-28.580000,NaN,parking,NaN,parking,c,201902,575.950134,78606.601562,False,True,10719


### Decisions

- We delete dups for main analysis and do robustness check withouth deleting them
- We tread cases where both descriptions are `<mdbremoved>` no different from others, even though it's somewhat more likely that they are genuinely different transactions

In [42]:
df = df.drop_duplicates(subset=dup_subset)

## Type 2 dups

### Definition

- `['user_id', 'date', 'amount', 'account_id']` are identical, `desc` is different, but `desc` in one txn is a truncated or slightly changed version of the other.

- FB reasons: same as above possible, but less likely. Seems more likely that they are actual duplicates created by an updating process (e.g. newer version of a txn with less redacted `desc` gets added without old version being removed, and `updated_flag` is incorrect.



In [43]:
dup_subset = ['user_id', 'date', 'amount', 'account_id']
dup_var = 'dup2'

df[dup_var] = df.duplicated(subset=dup_subset)

### Prevalence and value

How prevalent are duplicates?

In [44]:
n_df = len(df)
n_dups = len(df[df[dup_var]])
n_users_dups = df[df[dup_var]].user_id.nunique()
n_users_df = df.user_id.nunique()
txt = 'About {:.1%} of transactions across {:.0%} of users are potential dups.'
print(txt.format(n_dups / n_df, n_users_dups / n_users_df))

About 1.9% of transactions across 99% of users are potential dups.


Gross value of duplicated txns

In [45]:
gross_value = df[df[dup_var]].set_index('user_id').amount.abs().groupby('user_id').sum()
distr(gross_value)

count       424.00
mean       2497.45
std        8311.57
min           3.00
1%           11.08
5%           48.28
10%         104.04
25%         298.47
50%         880.35
75%        2097.92
90%        4584.54
95%        6842.71
99%       25811.31
max      106598.39
Name: amount, dtype: float64

Most frequent txns description

In [46]:
df[df[dup_var]].desc.value_counts(dropna=False)[:10]

<mdbremoved>                       674
daily od fee                       326
tfl travel ch     tfl.gov.uk/cp    225
tfl.gov.uk/cp     tfl travel ch    211
<mdbremoved>                       210
<mdbremoved> atm                   160
daily od fee chg                   133
islands news      london           127
<mdbremoved> - s/o                  97
gardens food and wlondon            87
Name: desc, dtype: int64

Most frequent auto tag

In [47]:
df[df[dup_var]].tag_auto.value_counts(dropna=False)[:10]

NaN                           4634
cash                          2895
transfers                     2739
bank charges                  2310
food, groceries, household    1542
public transport              1253
gambling                      1110
enjoyment                     1078
lunch or snacks                652
dining or going out            506
Name: tag_auto, dtype: int64

Proportion of txns per auto tag that are duplicated

In [48]:
txns_per_tag_overall = df.tag_auto.value_counts(dropna=False)
txns_per_tag_duplicated = df[df[dup_var]].tag_auto.value_counts(dropna=False) 
p_dup_per_tag = (txns_per_tag_duplicated / txns_per_tag_overall)
p_dup_per_tag.sort_values(ascending=False)[:10]

repayments            0.303867
legal                 0.175299
saving (general)      0.125000
vet                   0.104603
investment - other    0.102564
gambling              0.094977
media bundle          0.088889
bank charges          0.078101
social club           0.076923
isa                   0.066845
Name: tag_auto, dtype: float64

### Inspect dups

In [50]:
dup_txn_sample(df, dup_subset, n=2, seed=None)

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,savings,dup1,dup2,group
74671,527461874,2019-03-22,35177,20.0,cash notemac mar22 - one click se@xx:xx - atm,NaN,spend,other_spend,False,xxxx 0,2014-02-14,20k to 30k,1990.0,2017-05-25,724235,2020-08-14 20:59:00,hsbc,current,2019-03-23,2019-05-06,True,844.299988,personal,cash,NaN,cash,u,201903,200.301453,79785.007812,False,False,False,1756
74673,806078460,2019-03-22,35177,20.0,<mdbremoved> atm,NaN,spend,other_spend,False,xxxx 0,2014-02-14,20k to 30k,1990.0,2017-05-25,724235,2020-08-14 20:59:00,hsbc,current,2020-08-12,1900-01-01,True,844.299988,personal,cash,NaN,cash,c,201903,200.301453,79785.007812,False,False,True,1756
607714,191201530,2017-01-30,373777,50.0,"cash withdrawal at note machine atm morley service, leeds,50.00 gbp , on xx-xx-2017",NaN,spend,other_spend,False,ls27 8,2017-03-07,NaN,1963.0,2017-03-07,680911,2017-06-28 22:55:00,santander,current,2017-03-08,2017-10-23,True,NaN,personal,cash,NaN,cash,u,201701,NaN,29253.576563,False,False,False,12750
607716,191201527,2017-01-30,373777,50.0,"direct debit payment to h3g ref xxxxxxxxxxxxxx0117, mandate no 0013",hutchison 3g,spend,communication,False,ls27 8,2017-03-07,NaN,1963.0,2017-03-07,680911,2017-06-28 22:55:00,santander,current,2017-03-08,1900-01-01,True,NaN,hutchison 3g,mobile,NaN,mobile,c,201701,NaN,29253.576563,False,False,True,12750
607718,191201533,2017-01-30,373777,50.0,"direct debit payment to h3g ref xxxxxxxxxxxxxx0117, mandate no 0003",hutchison 3g,spend,communication,False,ls27 8,2017-03-07,NaN,1963.0,2017-03-07,680911,2017-06-28 22:55:00,santander,current,2017-03-08,1900-01-01,True,NaN,hutchison 3g,mobile,NaN,mobile,c,201701,NaN,29253.576563,False,False,True,12750


In [51]:
import difflib
import functools
import collections

from fuzzywuzzy import fuzz

DescAndId = collections.namedtuple('DescAndID', ['desc', 'id'])
longest_first = functools.partial(sorted, key=lambda x: len(x.desc), reverse=True)

def similarity_score(group):
    """Return similarity score between longest string in group and all others."""
    cols = list(group.columns)
    group['score_difflib'] = np.nan
    group['score_fuzz'] = np.nan
    items = [DescAndId(*item) for item in zip(group.desc, group.id)]
    longest, *others = longest_first(items)
    for o in others:
        group.loc[group.id == o.id, 'score_difflib'] = difflib.SequenceMatcher(None, longest.desc, o.desc).ratio()
        group.loc[group.id == o.id, 'score_fuzz'] = fuzz.partial_ratio(longest.desc, o.desc)
    return group[['score_difflib', 'score_fuzz'] + cols]

In [57]:
dd = dup_txn_sample(df, dup_subset, n=1, seed=None).groupby('group').apply(similarity_score)
dd

,score_difflib,score_fuzz,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,savings,dup1,dup2,group
918495,0.653061,67.0,558421618,2019-05-29,490277,3.0,waitrose 732 cd 6728 deb,waitrose,spend,household,True,n16 6,2019-02-11,NaN,1992.0,2019-02-11,1188710,2020-03-09 21:08:00,lloyds bank,current,2019-05-30,1900-01-01,True,473.660004,waitrose,"food, groceries, household",NaN,"food, groceries, household",c,201905,962.689209,17570.730469,False,False,False,17125
918496,NaN,NaN,558421619,2019-05-29,490277,3.0,tfl travel ch cd 6728 deb,tfl,spend,travel,True,n16 6,2019-02-11,NaN,1992.0,2019-02-11,1188710,2020-03-09 21:08:00,lloyds bank,current,2019-05-30,1900-01-01,True,473.660004,tfl,public transport,NaN,public transport,c,201905,962.689209,17570.730469,False,False,True,17125


- lowest same: .82 most, 72
- highest different: .71

Cases for decision:
- One mdbremoved, the other unremoved has very low score -> treat as dup or not? (could use 100 fuzz score to exclude)
- daily od fee _date_ only differs in date and has very high score but is different - exclude manually or handle automatically. some other descs that contain differing dates. could match dates and ensure they differ
- remove `-`, `)` and `(`

In [305]:
dd.desc.str.strip

<bound method StringMethods.normalize of <pandas.core.strings.accessor.StringMethods object at 0x17c3c1490>>

In [51]:
dup_subset = ['user_id', 'date', 'amount', 'account_id']
dup_var = 'dup2'
df[dup_var] = df.duplicated(subset=dup_subset)

In [101]:
from fuzzywuzzy import fuzz
from functools import partial

longest_first = partial(sorted, key=lambda x: len(x), reverse=True)

for idx, data in dd.groupby('group'):
    longest, *others = longest_first(data.desc.values)
    print(longest)
    for other in others:
        print('   {}'.format(other))
        print('   {}'.format(fuzz.partial_ratio(longest, other)), end='\n\n')


lnk sk store, 44/4 cd 8050 12jul15
   sby tamworth cd 8050 13jul15
   68

lnk sk store, 44/4 cd 8050 13dec15
   lnk star news coto cd 9447 12dec15
   59

the boathouse bras cd 4720 deb
   tesco stores 6711 cd 4720 deb
   62

<mdbremoved>
   <mdbremoved>
   100

   <mdbremoved>
   100

<mdbremoved> xxxxxx xxxx5560
   <mdbremoved>
   100

bank credit <mdbremoved>
   bank credit <mdbremoved>
   100

xxxxxx xxxx0290 internet transfer
   xxxxxx xxxx8658 internet transfer
   88

32 red cd 7512 deb
   32 red cd 7512 deb
   100

non-stg purch fee cd 6710 deb
   non-stg purch fee cd 6710 deb
   100

   non-stg purch fee cd 6710 deb
   100

   non-stg purch fee cd 6710 deb
   100

card payment to iz *canopy market,2.00 gbp, rate 1.00/gbp on 10-07-2020
   card payment to iz *crosstown,2.00 gbp, rate 1.00/gbp on 10-07-2020
   85



## Type 1 dups